In [10]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [11]:
def calculate_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [12]:
class CustomDataLoader:
    def __init__(self, features, labels, batch_size=1, validation_size=0.0, shuffle=False):

        if validation_size > 0:
            train_data, val_data, train_labels, val_labels = train_test_split(features, labels, test_size=validation_size, stratify=labels, random_state=42)
            train_data_tensor = torch.tensor(train_data).float().to(device)
            train_labels_tensor = torch.tensor(train_labels).long().to(device)
            val_data_tensor = torch.tensor(val_data).float().to(device)
            val_labels_tensor = torch.tensor(val_labels).long().to(device)
    
            train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
            val_dataset = TensorDataset(val_data_tensor, val_labels_tensor)

            self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
            self.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        else:
            features_tensor = torch.tensor(features).float().to(device)
            labels_tensor = torch.tensor(labels).long().to(device)

            dataset = TensorDataset(features_tensor, labels_tensor)

            self.train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
            self.val_loader = None

    def get_train_loader(self):
        return self.train_loader
    
    def get_val_loader(self):
        return self.val_loader

In [13]:
def evaluate_model(model, custom_train_loader, criterion, optimizer):
    num_epochs = 10000
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in custom_train_loader.get_train_loader():        
            optimizer.zero_grad()
            outputs = model(inputs.view(-1, 40))

            loss = criterion(outputs, labels, model)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        unreg_criterion = nn.CrossEntropyLoss()
        if (epoch + 1) % 1 == 0:
            print(f'Epoch {epoch+1}, Loss: {running_loss / (len(custom_train_loader.get_train_loader()))}')

            model.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in custom_train_loader.get_val_loader():
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs.view(-1, 40))
                    val_loss = unreg_criterion(outputs, labels)
                    running_val_loss += val_loss.item()

                avg_val_loss = running_val_loss / (len(custom_train_loader.get_val_loader()))
                print(f'Validation Loss: {avg_val_loss}')
            
            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.get_val_loader())
            print(f'Val Accuracy: {val_accuracy:.4f}, Val F1-score: {val_f1:.4f}')
            print()

    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in custom_train_loader.get_val_loader():
            outputs = model(inputs.view(-1, 40))
            val_loss = criterion(outputs, labels)
            running_val_loss += val_loss.item()

    avg_val_loss = running_val_loss / len(custom_train_loader.get_val_loader())
    print(f'Validation Loss: {avg_val_loss}')

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
data = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
data = data.drop(columns=["customerID"])
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data = data.dropna()

categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if data[col].nunique() == 2:
        data[col], _ = pd.factorize(data[col])

categorical_cols = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaymentMethod'])

In [16]:
X = data.drop(
    [
        "Churn",
    ],
    axis=1,
)
y = data["Churn"]

In [17]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [18]:
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, batch_size=16, validation_size=0.2, shuffle=True)

In [19]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, mode=""):
        super(CustomLinearLayer, self).__init__()
        self.linear = nn.Linear(input_size, output_size, bias=True)    
        
        nn.init.zeros_(self.linear.bias)
        if mode == "init_zero":
            nn.init.zeros_(self.linear.weight)
        elif mode == "split_inputs":
            self.split_inputs()
        else:
            self.custom_weight_init()

    def custom_weight_init(self):
        with torch.no_grad():
            rows, cols = self.linear.weight.size()
            weight = torch.zeros(rows, cols)
            for i in range(min(rows, cols) // 2):
                weight[2 * i, 2 * i] = 1
                weight[2 * i, 2 * i + 1] = -1
                if 2 * i + 1 < rows and 2 * i + 1 < cols:
                    weight[2 * i + 1, 2 * i] = -1
                    weight[2 * i + 1, 2 * i + 1] = 1
            self.linear.weight.copy_(weight)
            nn.init.zeros_(self.linear.bias)
            
    def split_inputs(self):
        weight = torch.zeros((self.linear.out_features, self.linear.in_features))
        
        for i in range(self.linear.out_features):
            if i % 2 == 0:
                weight[i, (i // 2) % self.linear.in_features] = 1
            else:
                weight[i, (i - 1) // 2 % self.linear.in_features] = -1
        
        self.linear.weight = nn.Parameter(weight)

    def forward(self, x):
        return self.linear(x)

In [72]:
class PairwiseCustomActivationNetwork(nn.Module):
    def __init__(self, input_size, num_layers, output_size):
        super(PairwiseCustomActivationNetwork, self).__init__()
        
        self.activation = nn.Softplus()
        
        layer_size = input_size * 2
        self.res_layer = CustomLinearLayer(input_size, layer_size, mode="split_inputs")
        self.layers = nn.ModuleList()

        for i in range(num_layers):
            self.layers.append(CustomLinearLayer(layer_size, layer_size))
            layer_size *= 2
            
        self.last_layer = CustomLinearLayer(layer_size, output_size, mode="init_zero")

    def forward(self, x):
        outputs = [self.activation(self.res_layer(x))]
        
        for layer in self.layers:
            concatenated_outputs = torch.cat(outputs, dim=1)
            outputs.append(self.activation(layer(concatenated_outputs)))

        concatenated_outputs = torch.cat(outputs, dim=1)
        return self.last_layer(concatenated_outputs)
    
model = PairwiseCustomActivationNetwork(40, 6, 2).to(device)
criterion = nn.CrossEntropyLoss()
print(summary(model, input_size=(1, 40)))

# remember to add original input features to every layer have the neuron be like [1, 0, 0], [-1, 0, 0], [0, 1, 0], [0, -1, 0]
# look at NN vs tree notebook
# or may be not

Layer (type:depth-idx)                   Output Shape              Param #
PairwiseCustomActivationNetwork          [1, 2]                    --
├─CustomLinearLayer: 1-1                 [1, 80]                   --
│    └─Linear: 2-1                       [1, 80]                   3,280
├─Softplus: 1-2                          [1, 80]                   --
├─ModuleList: 1-13                       --                        (recursive)
│    └─CustomLinearLayer: 2-2            [1, 80]                   --
│    │    └─Linear: 3-1                  [1, 80]                   6,480
├─Softplus: 1-4                          [1, 80]                   --
├─ModuleList: 1-13                       --                        (recursive)
│    └─CustomLinearLayer: 2-3            [1, 160]                  --
│    │    └─Linear: 3-2                  [1, 160]                  25,760
├─Softplus: 1-6                          [1, 160]                  --
├─ModuleList: 1-13                       --              

In [73]:
class CustomLoss(nn.Module):
    def __init__(self, l1_lambda):
        super(CustomLoss, self).__init__()
        self.l1_lambda = l1_lambda
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, outputs, labels, model):
        loss = self.cross_entropy_loss(outputs, labels)

        return loss
    
criterion = CustomLoss(0.001 * 0.1 * 10.0)

In [74]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
optimizer = optim.SGD(model.parameters(), lr=0.0001)
evaluate_model(model, custom_train_loader, criterion, optimizer)

Epoch 1, Loss: 0.4695108407177031
Validation Loss: 0.4457737561315298
Val Accuracy: 0.7889, Val F1-score: 0.7801

Epoch 2, Loss: 0.428804059301249
Validation Loss: 0.43925589831037953
Val Accuracy: 0.7925, Val F1-score: 0.7881

Epoch 3, Loss: 0.42304384972984815
Validation Loss: 0.43732415817000647
Val Accuracy: 0.7974, Val F1-score: 0.7946

Epoch 4, Loss: 0.42068568543023005
Validation Loss: 0.43623795749788935
Val Accuracy: 0.7982, Val F1-score: 0.7954

Epoch 5, Loss: 0.4193577145183967
Validation Loss: 0.43543119728565216
Val Accuracy: 0.8017, Val F1-score: 0.7989

Epoch 6, Loss: 0.4184817880053412
Validation Loss: 0.4347681672396985
Val Accuracy: 0.7996, Val F1-score: 0.7970

Epoch 7, Loss: 0.41784746445376764
Validation Loss: 0.4342036848379807
Val Accuracy: 0.8010, Val F1-score: 0.7983

Epoch 8, Loss: 0.41735779178667476
Validation Loss: 0.43371597067876294
Val Accuracy: 0.8017, Val F1-score: 0.7991

Epoch 9, Loss: 0.41696095919575205
Validation Loss: 0.4332908940586177
Val Accur

KeyboardInterrupt: 

In [75]:
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f"Layer: {name} | Weights: {param.data}")

Layer: res_layer.linear.weight | Weights: tensor([[ 9.9999e-01, -7.0296e-06,  7.8543e-06,  ...,  2.7559e-05,
         -2.5271e-05,  4.4993e-06],
        [-1.0000e+00,  4.3546e-06, -3.3533e-06,  ..., -9.4785e-06,
          5.5113e-06,  3.9543e-06],
        [-8.6647e-06,  1.0000e+00,  2.8408e-05,  ...,  2.3260e-05,
         -8.1537e-06,  4.5484e-05],
        ...,
        [ 6.7579e-05, -1.0137e-04, -4.8738e-05,  ...,  1.5827e-05,
         -1.0000e+00, -2.7119e-05],
        [ 1.6327e-05, -2.8739e-05, -2.4351e-05,  ...,  5.5670e-06,
         -6.5580e-07,  9.9998e-01],
        [-6.3283e-06,  7.8899e-06,  9.2957e-06,  ..., -4.6058e-06,
          6.4861e-06, -9.9999e-01]])
Layer: layers.0.linear.weight | Weights: tensor([[ 9.9999e-01, -1.0000e+00, -3.2534e-06,  ...,  4.5872e-06,
          6.6648e-07, -1.5862e-06],
        [-1.0000e+00,  1.0000e+00, -5.1788e-07,  ..., -3.1119e-06,
         -7.1759e-07, -2.6909e-06],
        [-3.0179e-06,  1.2876e-06,  1.0000e+00,  ...,  8.4481e-07,
          1.